In [2]:
from pymongo import MongoClient
import json
import pandas as pd

In [3]:
client = MongoClient('localhost',27017)

In [4]:
db = client.spotify

In [5]:
db.drop_collection(albums_raw)
db.drop_collection(artists_raw)
db.drop_collection(tracks_raw)

NameError: name 'albums_raw' is not defined

# Load

In [6]:
with open("kworb/spotify_artist_top10_albums_20251120.json") as f:
    albums = json.load(f)
db.albums_raw.insert_many(albums)

InsertManyResult([ObjectId('692cb8a7c9157c014a40864d'), ObjectId('692cb8a7c9157c014a40864e'), ObjectId('692cb8a7c9157c014a40864f'), ObjectId('692cb8a7c9157c014a408650'), ObjectId('692cb8a7c9157c014a408651'), ObjectId('692cb8a7c9157c014a408652'), ObjectId('692cb8a7c9157c014a408653'), ObjectId('692cb8a7c9157c014a408654'), ObjectId('692cb8a7c9157c014a408655'), ObjectId('692cb8a7c9157c014a408656'), ObjectId('692cb8a7c9157c014a408657'), ObjectId('692cb8a7c9157c014a408658'), ObjectId('692cb8a7c9157c014a408659'), ObjectId('692cb8a7c9157c014a40865a'), ObjectId('692cb8a7c9157c014a40865b'), ObjectId('692cb8a7c9157c014a40865c'), ObjectId('692cb8a7c9157c014a40865d'), ObjectId('692cb8a7c9157c014a40865e'), ObjectId('692cb8a7c9157c014a40865f'), ObjectId('692cb8a7c9157c014a408660'), ObjectId('692cb8a7c9157c014a408661'), ObjectId('692cb8a7c9157c014a408662'), ObjectId('692cb8a7c9157c014a408663'), ObjectId('692cb8a7c9157c014a408664'), ObjectId('692cb8a7c9157c014a408665'), ObjectId('692cb8a7c9157c014a4086

In [7]:
with open("kworb/spotify_monthly_listeners_20251120.json") as f:
    artist_listeners = json.load(f)
db.artists_raw.insert_many(artist_listeners)

InsertManyResult([ObjectId('692cb8a9c9157c014a408a13'), ObjectId('692cb8a9c9157c014a408a14'), ObjectId('692cb8a9c9157c014a408a15'), ObjectId('692cb8a9c9157c014a408a16'), ObjectId('692cb8a9c9157c014a408a17'), ObjectId('692cb8a9c9157c014a408a18'), ObjectId('692cb8a9c9157c014a408a19'), ObjectId('692cb8a9c9157c014a408a1a'), ObjectId('692cb8a9c9157c014a408a1b'), ObjectId('692cb8a9c9157c014a408a1c'), ObjectId('692cb8a9c9157c014a408a1d'), ObjectId('692cb8a9c9157c014a408a1e'), ObjectId('692cb8a9c9157c014a408a1f'), ObjectId('692cb8a9c9157c014a408a20'), ObjectId('692cb8a9c9157c014a408a21'), ObjectId('692cb8a9c9157c014a408a22'), ObjectId('692cb8a9c9157c014a408a23'), ObjectId('692cb8a9c9157c014a408a24'), ObjectId('692cb8a9c9157c014a408a25'), ObjectId('692cb8a9c9157c014a408a26'), ObjectId('692cb8a9c9157c014a408a27'), ObjectId('692cb8a9c9157c014a408a28'), ObjectId('692cb8a9c9157c014a408a29'), ObjectId('692cb8a9c9157c014a408a2a'), ObjectId('692cb8a9c9157c014a408a2b'), ObjectId('692cb8a9c9157c014a408a

In [8]:
with open("kworb/spotify_us_daily_artists_20251120.json") as f:
    tracks = json.load(f)
db.tracks_raw.insert_many(tracks)

InsertManyResult([ObjectId('692cb8b0c9157c014a4093d7'), ObjectId('692cb8b0c9157c014a4093d8'), ObjectId('692cb8b0c9157c014a4093d9'), ObjectId('692cb8b0c9157c014a4093da'), ObjectId('692cb8b0c9157c014a4093db'), ObjectId('692cb8b0c9157c014a4093dc'), ObjectId('692cb8b0c9157c014a4093dd'), ObjectId('692cb8b0c9157c014a4093de'), ObjectId('692cb8b0c9157c014a4093df'), ObjectId('692cb8b0c9157c014a4093e0'), ObjectId('692cb8b0c9157c014a4093e1'), ObjectId('692cb8b0c9157c014a4093e2'), ObjectId('692cb8b0c9157c014a4093e3'), ObjectId('692cb8b0c9157c014a4093e4'), ObjectId('692cb8b0c9157c014a4093e5'), ObjectId('692cb8b0c9157c014a4093e6'), ObjectId('692cb8b0c9157c014a4093e7'), ObjectId('692cb8b0c9157c014a4093e8'), ObjectId('692cb8b0c9157c014a4093e9'), ObjectId('692cb8b0c9157c014a4093ea'), ObjectId('692cb8b0c9157c014a4093eb'), ObjectId('692cb8b0c9157c014a4093ec'), ObjectId('692cb8b0c9157c014a4093ed'), ObjectId('692cb8b0c9157c014a4093ee'), ObjectId('692cb8b0c9157c014a4093ef'), ObjectId('692cb8b0c9157c014a4093

# Transform

In [9]:
db.artists_raw.update_many(
    {},
    [
        {
            "$set": {
                "artist_id": {
                    "$arrayElemAt": [
                        {"$split": ["$artist_url", "/"]},
                        -1
                    ]
                }
            }
        }
    ]
)

UpdateResult({'n': 2500, 'nModified': 2500, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [10]:
db.tracks_raw.update_many(
    {},
    [
        {
            "$set": {
                "artist_id": {
                    "$arrayElemAt": [
                        {"$split": ["$artist_url", "/"]},
                        -1
                    ]
                }
            }
        }
    ]
)

UpdateResult({'n': 2700, 'nModified': 200, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [12]:
db.tracks_raw.update_many(
    {},
    [
        {
            "$set": {
                "daily_streams": {
                    "$toInt": {
                        "$replaceAll": {
                            "input": "$daily_streams",
                            "find": ",",
                            "replacement": ""
                        }
                    }
                }
            }
        }
    ]
)

UpdateResult({'n': 2700, 'nModified': 200, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [13]:
db.tracks_raw.update_many(
    {},
    [
        {
            "$set": {
                "daily_streams": {
                    "$toInt": "$daily_streams"
                }
            }
        }
    ]
)

UpdateResult({'n': 2700, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

# Google trends

In [ ]:
with open("google_trends_raw.json", "r") as f:
    google_raw = json.load(f)

google = pd.json_normalize(google_raw)
print(google.head())

In [ ]:
# Transformation if needed
google["date"] = pd.to_datetime(google["date"])
google["year_month"] = google["date"].google.to_period("M").astype(str)
google["state_name"] = google["state_name"].str.title()
google["interest_index"] = google["interest_index"].astype(int)
google["artist_name"] = google["artist_name"].str.strip()

In [ ]:
google = db["google_trends"]
records = df.to_dict("records")

google.insert_many(records)